In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS, Qdrant

In [ ]:
# ============================================
# 2. 문서 임베딩 & 벡터 저장
# ============================================
## 주의!! 생성만 함

def create_vector_store(documents: list[str], save_path: str = "vectorstore"):
    """
    문서를 임베딩하고 벡터 저장소 생성 (FAISS)
    
    Args:
        documents: 임베딩할 문서 리스트
        save_path: 저장 경로
    """
    # 텍스트 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    texts = text_splitter.create_documents(documents)
    
    # 임베딩 & 벡터 저장소 생성
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    vectorstore = FAISS.from_documents(texts, embeddings)
    
    # 저장
    vectorstore.save_local(save_path)
    print(f"✓ 벡터 저장소 저장 완료: {save_path}")
    
    return vectorstore


def load_vector_store(load_path: str = "vectorstore"):
    """저장된 벡터 저장소 로드 (FAISS)"""
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    vectorstore = FAISS.load_local(
        load_path, 
        embeddings,
        allow_dangerous_deserialization=True
    )
    print(f"✓ 벡터 저장소 로드 완료: {load_path}")
    return vectorstore

In [15]:
create_vector_store(['누구세요'])

✓ 벡터 저장소 저장 완료: vectorstore


In [16]:
def load_vector_store(load_path: str = "vectorstore"):
    """저장된 벡터 저장소 로드 (FAISS)"""
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    vectorstore = FAISS.load_local(
        load_path, 
        embeddings,
        allow_dangerous_deserialization=True
    )
    print(f"✓ 벡터 저장소 로드 완료: {load_path}")
    return vectorstore

In [17]:
vector_store = load_vector_store()

✓ 벡터 저장소 로드 완료: vectorstore


In [ ]:
# 문서 추가 저장

from langchain_core.documents import Document

new_docs = [
    Document(page_content="이건 새로운 문서예요.", metadata={"source": "manual"}),
    Document(page_content="추가 테스트 문서입니다.", metadata={"author": "이지"})
]

vector_store.add_documents(new_docs)

In [26]:
# docstore에 저장된 모든 문서 키 확인
for doc_id, doc in vector_store.docstore._dict.items():
    print(f"ID: {doc_id}")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print("-" * 50)

ID: e6364e50-5df1-4bfb-8913-3e4379441553
Content: 누구세요
Metadata: {}
--------------------------------------------------
ID: 11e5a099-ae00-4b39-bfb7-f3a80dc09ed7
Content: 이건 새로운 문서예요.
Metadata: {'source': 'manual'}
--------------------------------------------------
ID: 5fde1412-7c29-4944-a25c-8d0428ce1113
Content: 추가 테스트 문서입니다.
Metadata: {'author': '이지'}
--------------------------------------------------


In [30]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams